# 

In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/gsm8k"
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
inputs = sagemaker_session.upload_data(
    path="gsm8k.jsonl",
    bucket=bucket,
    key_prefix=prefix
)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-419599817388/sagemaker/gsm8k/gsm8k.jsonl


In [30]:
from sagemaker.pytorch import PyTorch
from pathlib import Path

estimator = PyTorch(
    entry_point='fine_tune.py',
    source_dir=f'{Path.cwd()}/src',
    role=role,
    py_version="py311",
    framework_version='2.3.0',
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters={
        "epochs": 10,
        "model-id": "unsloth/Llama-3.2-1B-Instruct",
        "lr": 1e-3,
        "data-file": "gsm8k.jsonl",
    },
    disable_output_compression=True,
)

In [31]:
estimator.fit({"training": inputs})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-05-16-15-58-34-571


2025-05-16 15:58:35 Starting - Starting the training job
2025-05-16 15:58:35 Pending - Training job waiting for capacity......
2025-05-16 15:59:39 Pending - Preparing the instances for training...
2025-05-16 16:00:05 Downloading - Downloading input data...
2025-05-16 16:00:30 Downloading - Downloading the training image.....................
2025-05-16 16:03:58 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 530.30.02
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
/opt/conda/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 4

## Move model artifacts for Custom Model Import

In [42]:
last_train_job = estimator.jobs[-1].describe()
artifact_path = last_train_job['ModelArtifacts']['S3ModelArtifacts']
artifact_key = artifact_path[artifact_path.find(bucket) + len(bucket):].lstrip('/')
print(f'Model Artifacts at {artifact_path}')

Model Artifacts at s3://sagemaker-us-east-1-419599817388/pytorch-training-2025-05-16-15-58-34-571/output/model


In [49]:
import boto3
import tarfile

sts_client = boto3.client('sts')
account_info = sts_client.get_caller_identity()
account_id = account_info['Account']

bucket_name = f"bedrock-custom-model-{account_id}"

In [48]:
s3 = boto3.resource('s3')
src_bucket = s3.Bucket(bucket)
dst_bucket = s3.Bucket(bucket_name)

for obj in src_bucket.objects.filter(Prefix=artifact_key):
	old_source = {'Bucket': bucket, 'Key': obj.key}
	new_key = obj.key.replace(artifact_key, 'fine-tuned-model', 1)
	print(f"Copy {obj.key} -> {new_key}")
	new_obj = dst_bucket.Object(new_key)
	new_obj.copy(old_source)

Copy pytorch-training-2025-05-16-15-58-34-571/output/model/config.json -> fine-tuned-model/config.json
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/generation_config.json -> fine-tuned-model/generation_config.json
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/model-00001-of-00003.safetensors -> fine-tuned-model/model-00001-of-00003.safetensors
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/model-00002-of-00003.safetensors -> fine-tuned-model/model-00002-of-00003.safetensors
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/model-00003-of-00003.safetensors -> fine-tuned-model/model-00003-of-00003.safetensors
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/model.safetensors.index.json -> fine-tuned-model/model.safetensors.index.json
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/special_tokens_map.json -> fine-tuned-model/special_tokens_map.json
Copy pytorch-training-2025-05-16-15-58-34-571/output/model/tokenizer.json